<a href="https://colab.research.google.com/github/dxbezerra/tvdi/blob/master/TVDI_projeto_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cálculo do TVDI para o estado do Ceará — uma estimativa de seca em Python**

Autores: Diego Xavier, Gustavo Nagel, Raíssa Teixeira, Stella Coelho. 

Disciplina de Introdução à Programação (SER-307) - 2019

Instituto Nacional de Pesquisas Espaciais (INPE)


---
O índice TVDI (Temperature-Vegetation Dryness Index) é um índice de estresse hídrico baseado na relação entre a temperatura da superfície e o índice de vegetação. Assim, ao integrar o NDVI (Normalized Difference Vegetation Index) e a temperatura de superfície, o TVDI é capaz de estimar o grau da seca de uma localidade, utilizando dados na faixa do visível, infra-vermelho próximo e termal (SANDHOLT et al, 2002). Este índice é de extrema importância para o monitoramento dos períodos de seca de uma região, e pode ser facilmente obtido com dados de sensoriamento remoto nas faixas espectrais citadas. 	
Neste sentido, o seguinte programa tem o objetivo de operacionalizar o monitoramento de seca agrícola para o estado do Ceará a partir de dados orbitais do sensor MODIS, apresentando um exemplo para a segunda quinzena de Setembro no ano de 2018, sendo um período de poucas chuvas no estado. O TVDI obtido pode ser assim aplicado para qualquer período em que estejam disponíveis os dados do sensor MODIS, e se combinado com dados de precipitação provenientes de estações meteorológicas é capaz de gerar informação extremamente relevante para uma localidade.

![alg](https://docs.google.com/uc?export=download&id=1x-u197nO0fc5QyZcH0Fr2W9hmWYM7wji)


As etapas para a obtenção do TVDI consistem de:
1.   Download das Imagens
2.   Extração das bandas de interesse das imagens
3.   Mosaicagem
4.   Recorte para área de interesse
5.   Acesso das imagens
6.   Tratamento dos dados
7.   Cálculo das retas
8.   Cálculo do TVDI





In [1]:
# Limpar dados, instalar bibliotecas e fazer download do shapefile
!pip install setuptools==40.6.3
!pip install pyModis --upgrade
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y --prefix /usr/local -c <<<your wish>>>>

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')

!rm -rf sample_data
!rm -rf processed
!rm -rf tvdi
!git clone https://github.com/dxbezerra/tvdi.git
!sudo pip install -U git+https://github.com/matplotlib/basemap.git


  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 573kB 2.0MB/s 
  Found existing installation: setuptools 38.4.0
    Uninstalling setuptools-38.4.0:
      Successfully uninstalled setuptools-38.4.0


  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 61kB 3.1MB/s 
    100% |████████████████████████████████| 583kB 1.9MB/s 
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-d99hi8br/GDAL/
--2019-05-24 15:32:36--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda, Inc.
installing: ca-certificates-2017.08.26-h1d4fec5_0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-hc7b2577_8 ...
installing: libgcc-ng-7.2.0-h7cc24e2_2 ...
installing: libgfortran-ng-7.2.0-h9f7466a_2

In [2]:
# Importar módulos
import gdal
from gdalconst import *
import numpy as np
from glob import glob
import scipy
from scipy.stats import gaussian_kde
from os import path as osp
import os, calendar, itertools, subprocess
from pymodis import downmodis
from pymodis.convertmodis_gdal import convertModisGDAL
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap

WxPython missing, no GUI enabled


ModuleNotFoundError: ignored

In [0]:
# Criar diretório dos dados
path = './raw_data'
if not osp.exists(path):
    os.mkdir(path)
print(os.listdir('./'))

# **1. Download das imagens**
A partir da biblioteca pyModis realizamos o download das imagens de interesse. No caso, utilizamos o produto MOD13A2 que corresponde ao índices de vegetação NDVI e EVI composto de 16 dias e o produto MOD11A2 correspondente a Temperature Superficial do Solo (TS) para de um intervalo de 8 dias.
O período analisado é referente ao dia juliano 257 (segunda quinzena de Setembro), que é o período mais seco no estado do Ceará.

In [0]:
# Usuário e senha Earthdata
user     = 'grupo_tvdi'
password = 'Ninguempodesaber1'

# Cenas e produtos a serem baixados
tiles    = 'h14v09','h13v09'
products = ['MOD13A2.006', 'MOD11A2.006']

# Período selecionado
day     = '2018-09-14'
enddate = '2018-09-29'

# Download
print('Realizando download...')

for p in products:
  modisDown = downmodis.downModis(destinationFolder=path,
  password=password, user=user, tiles=tiles, product=p, today=day,
  enddate=enddate)

  modisDown.connect()
  modisDown.downloadsAllDay()

print('Download completo.')

# Checkar imagens baixadas e o seu tamanho em megabytes
fls = sorted(glob(path + '/*.hdf'))
for f in fls:
  print(osp.basename(f),'-', osp.getsize(f)/10e5, 'MB')

# **2. Extração das bandas de interesse das imagens**
Tendo em vista que as imagens MODIS são contidas de diversas bandas (ex.: banda de qualidade do pixel, presença de nuvens, ângulo solar zenital, etc.), as bandas de interesse  são primeiramente extraídas com o módulo *convertModisGDAL*. Para o produto MOD13A2 utilizamos o NDVI e para o produto MOD11A2 a banda utilizada é a do TS capturado durante o dia (*LST_Day_1km*), ambos sendo a primeira banda.

In [0]:
# Selecionar as datas julianas únicas das imagens baixadas (np.unique seleciona apenas valores que não se repetem)
juliandays = np.unique([osp.basename(f).split(".")[1][-3:] for f in fls])

# Diretório de saída das bandas
path = './processed'
if not osp.exists(path):
  os.mkdir(path)

# Extrair bandas  
for f in fls:
  
  # Banda selecionada (Seleciona somente a primeira banda)
  subset = "1"
  
  # Extrair bandas com pymodis (obs: epsg é o código de reprojeção da imagem)
  fileout = osp.join(path, osp.basename(f)[:-4])
  extr = convertModisGDAL(f, fileout, subset, res=None, outformat='GTiff', epsg=4326)
  extr.run(quiet=True)
   
# Checkar bandas extraídas
fls = sorted(glob(path + '/*.tif'))
for f in fls:
  print(osp.basename(f))

#**3. Mosaicagem**
As imagens baixadas são correspondentes às cenas do MODIS (h14v09 e h13v09) e necessitam que sejam combinadas para o Estado do Ceará. Esse processo é realizado selecionando as imagens de mesmo produto e período, porém de cenas distintas. Mais adiante o GDAL é utilizado para criar o mosaico.

In [0]:
# Listas
products   = ['MOD13A2.006', 'MOD11A2.006']
juliandays = np.unique([osp.basename(f).split(".")[1][-3:] for f in fls])

# Selecionar imagens de mesmo produto e período, porém de cenas distintas
for p in products:
  p = p.split(".")[0]
  
  for jd in juliandays:

    # Selecionar todas as imagens criadas no passo anterior (bandas extraídas) para o mesmo produto
    glb = glob('./processed/{0}*.tif'.format(p))

    # Guardar imagens de cenas distintas de mesmo produto e de mesmo período em uma lista
    pair = []
    for im in glb:
      if jd in im:
        pair.append(im)
    
    # Se a lista de imagens não for vazia, prosseguir.
    if not pair == []:
    
      # Remover espaços no nome do arquivo (pra não dar problema com o GDAL)
      rm_spaces = pair[0].replace(" ", "")
      
      # Criar um novo nome para arquivo de saída
      fname = rm_spaces[:-4] + ".MOSAIC.vrt"

      # Criar mosaico com o GDAL utilizando o shell
      gdal.BuildVRT(fname, pair)
      translateCmd = 'gdal_translate -of GTiff {0} {1}.tif'.format(fname, fname[:-4])
      subprocess.call(translateCmd, shell=True)

# Checkar mosaicos criados
fls = sorted(glob(path + '/*MOSAIC.tif'))
for f in fls:
  print(osp.basename(f))

# **4. Recorte para área de interesse**
Para focarmos somente na área de nosso interesse, realizamos um recorte utilizando o arquivo *shapefile* dos limites estaduais do Ceará, evitando assim o processamento de dados irrelevantes posteriormente.

In [0]:
# Arquivo shapefile
shp = './tvdi/ce.shp'
print(os.path.isfile(shp))

# Realizar corte a partir dos mosaicos da pasta dos arquivos processados
for f in fls:
  
  # Nome do arquivo da imagem recortada
  fname = f[:-4] + ".CLIP.tif"

  # Realizar recorte com gdalwarp utilizando o shell do computador
  warpCmd = 'gdalwarp -dstnodata -3000 -cutline {0} {1} {2}\
            -crop_to_cutline -overwrite'.format(shp, f, fname)
  subprocess.call(warpCmd, shell=True)

# Checkar recortes criados
fls = sorted(glob(path + '/*CLIP.tif'))
for f in fls:
  print(osp.basename(f))

In [0]:
# Plot das imagens recortadas
dset_tss1, dset_tss2, dset_ndvi = gdal.Open(fls[0], GA_ReadOnly),\
                                  gdal.Open(fls[1], GA_ReadOnly),\
                                  gdal.Open(fls[2], GA_ReadOnly)

tss1_band, tss2_band, ndvi_band = dset_tss1.GetRasterBand(1),\
                                  dset_tss2.GetRasterBand(1),\
                                  dset_ndvi.GetRasterBand(1)

plt.figure(figsize=(12, 12))

plt.subplot(131)
plt.title("TS 1º 8 dias")
plt.imshow(tss1_band.ReadAsArray() * 0.02, cmap='magma', vmin=300, vmax=325)
plt.colorbar(orientation="horizontal", pad=0.05)

plt.subplot(132)
plt.title("TS 2º 8 dias")
plt.imshow(tss2_band.ReadAsArray() * 0.02, cmap='magma', vmin=300, vmax=325)
plt.colorbar(orientation="horizontal", pad=0.05)

plt.subplot(133)
plt.title("NDVI")
plt.imshow(ndvi_band.ReadAsArray() * 0.0001, cmap='RdYlGn', vmin=0.1, vmax=0.8)
plt.colorbar(orientation="horizontal", pad=0.05)

plt.tight_layout(pad=1)
plt.show()

dset_tss1, dset_tss2, dset_ndvi = None, None, None

# **5. Acesso as  imagens**
Para igualar o período observado entre as imagens de NDVI e TS, nesta seção é realizada uma média temporal das duas imagens TS de 8 dias. 

In [0]:
fls = sorted(glob(path + '/*CLIP.tif'))

# Carregar imagens de temperatura
dset_tss1 = gdal.Open(fls[0], GA_ReadOnly)
dset_tss2 = gdal.Open(fls[1], GA_ReadOnly)

dset_tss1_band = dset_tss1.GetRasterBand(1)
dset_tss2_band = dset_tss2.GetRasterBand(1)

# Multiplicar pelo fator de escala
dset_tss1_array = dset_tss1_band.ReadAsArray() * 0.02
dset_tss2_array = dset_tss2_band.ReadAsArray() * 0.02

# Criar máscara (substituir np.nan para valores de 0, que são os valores inválidos)
dset_tss1_array[dset_tss1_array == 0] = np.nan
dset_tss2_array[dset_tss2_array == 0] = np.nan

# Calcular média temporal ignorando nan
stack_2d = np.array([dset_tss1_array, dset_tss2_array])
TS      = scipy.nanmean(stack_2d, axis=0) # média: 311.56830648359954

# Fechar datasets
dset_tss1, dset_tss2 = None, None

# Arredondar para duas casas decimais o valor médio de TS
TS = np.round(TS, 2)

O acesso à imagem de NDVI segue da seguinte forma:

In [0]:
# Carregar imagem de NDVI
dset_ndvi = gdal.Open(fls[2], GA_ReadOnly)
ndvi_band = dset_ndvi.GetRasterBand(1)

# Armazenar informações
gt        = dset_ndvi.GetGeoTransform()
proj      = dset_ndvi.GetProjectionRef()
dtype     = ndvi_band.DataType

# Multiplicar pelo fator de escala
NDVI      = ndvi_band.ReadAsArray() * 0.0001

# Criar máscara (substituir np.nan para valores de 0, que são os valores inválidos)
NDVI[NDVI == -0.3] = np.nan

# Fechar dataset
dset_ndvi = None

# Arredondar para duas casas decimais e printar valor médio de NDVI
NDVI = np.round(NDVI, 2)

Visualização das imagens escaladas e histogramas:

In [0]:
# Criar diretório dos dados finais
path = './output_final'
if not osp.exists(path):
    os.mkdir(path)

# Tamanho do plot
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(12,12))
plt.subplots_adjust(left=0.125, right = 0.9, wspace=0.3)

# Subplot TS
img1 = ax1.imshow(TS, cmap='magma', vmin=295, vmax=325)
ax1.set_title("TS - 16 dias ")
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes("right", size="5%", pad=0.2)
cbar = fig.colorbar(img1, cax=cax1)
cbar.set_label("Temperatura (Kelvin)")

# Subplot NDVI
img2 = ax2.imshow(NDVI, cmap='RdYlGn', vmin=0.1, vmax=0.8)
ax2.set_title("NDVI - 16 dias")
divider = make_axes_locatable(ax2)
cax2 = divider.append_axes("right", size="5%", pad=0.2)
cbar = fig.colorbar(img2, cax=cax2)
cbar.set_label("NDVI")

# Subplot Histograma TS
img3 = ax3.hist(TS.ravel(), bins=256, range=(295, 325), lw=4, ec='royalblue')
ax3.set_aspect(1./ax3.get_data_ratio())
ax3.set_title("Histograma TS")
ax3.set_xlabel('TS')
ax3.set_ylabel('No. de pixels')

# Subplot Histograma NDVI
img4 = ax4.hist(NDVI.ravel(), bins=256, range=(0.1, 0.8), lw=4, ec='royalblue')
ax4.set_aspect(1./ax4.get_data_ratio())
ax4.set_title("Histograma NDVI")
ax4.set_xlabel('NDVI')
ax4.set_ylabel('No. de pixels')

# Remover bordas da figura
fig.show()

# **6. Tratamento dos dados**
Antes da extração do TVDI, os valores de TS fora de 3 desvios padrões foram removidos para melhor representação dos limiares úmido e seco. Além disso, os valores de NDVI menores que zero foram eliminados pois tendem a ser representativos de corpos d'àgua.

Eliminar outliers

In [0]:
# Retirar valores fora de 3 desvios padrões da imagem de TS
std  = np.nanstd(TS) # desvio padrão
mean = np.nanmean(TS) # média

lower_limit = mean - (std*3)
upper_limit = mean + (std*3)

np.where(TS, TS < lower_limit, np.nan)
np.where(TS, TS > upper_limit, np.nan)

print("Limiar mínimo:", np.nanmin(TS), "K")
print("Limiar máximo:", np.nanmax(TS), "K")

# Retirar valores menores que zero na imagem de NDVI
NDVI[NDVI < 0] = np.nan

# **7. Cálculo das retas**

Para o cálculo das retas, primeiramente são determinadas as temperaturas correspondentes ao mesmo valor de NDVI. Então são estabelecidas as temperaturas máxima e mínima para cada NDVI.

![TVDI](https://docs.google.com/uc?export=download&id=1A81JkeXKesyWRklvICWCuh0bsOey7MGE)Fonte: Du et al. (2017)



Retirada dos valores máximos e mínimos de TS

In [0]:
# Listas para mínimos (limite úmido) e máximos de TS (limite seco)
MiniList    = []
MaxList     = []
# Criar um vetor de NDVI (0 a 1 com espaçamento de 0.01)
NDVI_vector = np.round(np.arange(0.01, 1.01, 0.01), 2)

# Primeiramente são encontrados os valores de TS para mesmo NDVI
for val in NDVI_vector:
  TS_vector_val = []
  row, col = np.where(NDVI == val) # extrair index
  
  # Com a localização destes NDVIs, retiramos os valores de temperatura
  # correspondentes a estas posições (linhas e colunas)
  for i in range(len(row)):
    if np.isfinite(TS[row[i], col[i]]):
      TS_vector_val += [TS[row[i], col[i]]]
  
  # Se houver valores de TS para o NDVI desejado, é retirado os
  # valores máximo e mínimo
  if TS_vector_val != []:
    TS_min_val = np.min(TS_vector_val)
    TS_max_val = np.max(TS_vector_val)
  else:
    TS_min_val = np.nan
    TS_max_val = np.nan

  # Os valores encontrados são adicionados na listas MiniList e MaxList
  MiniList += [TS_min_val]
  MaxList  += [TS_max_val]
  
print("Valores mínimos", "- len:", len(MiniList))
print(MiniList)
print("Valores máximos", "- len:", len(MaxList))
print(MaxList)
print("Valores de NDVI", "- len:", len(NDVI_vector))
print(list(NDVI_vector))

Retirar coeficientes *a* e *b* para cálculo do TVDI

In [0]:
# Reta minimo
MiniList_fin = []
NDVI_fin = []

for i, val in enumerate(MiniList):
  if np.isfinite(val):
    MiniList_fin += [val]
    NDVI_fin += [NDVI_vector[i]]
print(MiniList_fin)
print(NDVI_fin)

  # Retirar coeficientes
MinPfit = np.polyfit(NDVI_fin[17:], MiniList_fin[17:], 1)
print(MinPfit)

# Reta maximo
MaxList_fin = []
NDVI_fin = []
for i, val in enumerate(MaxList):
  if np.isfinite(val):
    MaxList_fin += [val]
    NDVI_fin += [NDVI_vector[i]]
print(MaxList_fin)
print(NDVI_fin)


  # Retirar coeficientes
MaxPfit = np.polyfit(NDVI_fin[17:], MaxList_fin[17:], 1)
print(MaxPfit)

Visualizar gráfico de dispersão

In [0]:
# gerar o primeiro e o último pontos do limite úmido e limite seco
a1, b1 = MaxPfit
a2, b2 = MinPfit
linhamax = [b1 + (a1 * 0), b1 + (a1 * 1)]
linhamin = [b2 + (a2 * 0), b2 + (a2 * 1)]

plt.figure(figsize=(10,7))
plt.plot(NDVI.ravel(), TS.ravel(), "+", color='black')#, markersize=3)
plt.plot(NDVI_vector[21:], MiniList[21:], '+', color='b')
plt.plot(NDVI_vector[21:], MaxList[21:], '+', color='r')
plt.plot([0, 1], linhamax, color='r', markersize=8)
plt.plot([0, 1], linhamin, color='b', markersize=8)

plt.xlabel("NDVI")
plt.ylabel("TS")
plt.title("NDVI vs TS Scatterplot")

In [0]:
# Scatterplot de densidade
NDVIlista = []
Templista = []
shape = np.shape(NDVI)
for b in range(shape[0]): # linhas
  for c in range(shape[1]): # Colunas
    if np.isfinite(TS[b][c]) and np.isfinite(NDVI[b][c]):
      NDVIlista += [NDVI[b][c]]
      Templista += [TS[b][c]]
print(NDVIlista)
print(Templista)

xy = np.vstack([NDVIlista, Templista])
z = gaussian_kde(xy)(xy)

In [0]:
fig, ax = plt.subplots()
ax.scatter(NDVIlista, Templista, c=z, s=8, edgecolor='')
plt.show()

# **8. Cálculo do TVDI**

![TVDI](https://docs.google.com/uc?export=download&id=1nh-vaYSW5A-g5N2uour3zTiABSmvv86z)

O índice calcula a proporção entre a temperatura do pixel em questão e  os limites seco e úmido representados pelas retas. O TVDI varia de 0 a 1, sendo que quanto mais próximo de 1 maior é o nível de seca da região.

In [0]:
a1, b1 = MaxPfit
a2, b2 = MinPfit
print(a1,b1)
print(a2,b2)

Ts_max = b1 + (a1 * NDVI)
Ts_min = b2 + (a2 * NDVI)

TVDI = (TS - Ts_min) / (Ts_max - Ts_min)

print(np.nanmin(TVDI))
print(np.nanmax(TVDI))

Salvar imagem do TVDI em formato GeoTIFF



In [0]:
# Gerar arquivo GeoTIFF
fname_out   = './output_final/TVDI_ce.tif'
driver      = gdal.GetDriverByName('GTiff')
data_type   = ndvi_band.DataType
dset_output = driver.Create(fname_out, NDVI.shape[1], NDVI.shape[0], 1, gdal.GDT_Float32)
dset_output.SetGeoTransform(gt)
dset_output.SetProjection(proj)
dset_output.GetRasterBand(1).WriteArray(TVDI)
dset_output.FlushCache()
dset_output = None

Plotar imagem e histograma do TVDI

In [0]:
# Carregar imagem do TVDI gerada
dataset   = gdal.Open("./output_final/TVDI_ce.tif")
im_array  = dataset.GetRasterBand(1).ReadAsArray()
msk_array = np.ma.masked_invalid(im_array)

# Retirar matrizes lat lon
xy = ds.GetGeoTransform() 
x, y = ds.RasterXSize, ds.RasterYSize    
lon_start = xy[0] 
lon_stop  = x*xy[1]+xy[0] 
lon_step  = xy[1]    
lat_start = xy[3] 
lat_stop  = y*xy[5]+xy[3] 
lat_step  = xy[5]

xx = np.arange(lon_start, lon_stop, lon_step) 
yy = np.arange(lat_start, lat_stop, lat_step)    
lons, lats = np.meshgrid(xx,yy)

# Desenhar meridianos e pararelos
plt.figure(figsize=(8,7))

m = Basemap(projection='merc', llcrnrlat=-8, urcrnrlat=-2,\
            llcrnrlon=-42, urcrnrlon=-37, resolution='l', suppress_ticks=True)

parallels = np.arange(-80., 0, 1)
m.drawparallels(parallels, labels=[True, False, False, False], linewidth=0.25)
meridians = np.arange(-350., 10, 1)
m.drawmeridians(meridians, labels=[False, False, False, True], linewidth=0.25)

# Plotar TVDI
m.readshapefile("./tvdi/limites_estaduais_bra", 'limites', linewidth=1)
cm = m.pcolormesh(lons, lats, msk_array, vmin = 0., vmax = 1., latlon=True, cmap='CMRmap')

# Adicionar colorbar
cbar = m.colorbar(cm, location='right', pad="5%")
cbar.set_label('TVDI')
plt.title('TVDI', fontsize=14)

# Salvar figura
plt.tight_layout()
plt.show()
print("Valores próximos a 1 indicam condições secas e próximo a 0 condições úmidas")

In [0]:
# Plot do histograma
plt.figure(figsize=(6,5))
plt.hist(TVDI.ravel(), bins=256, range=(0, 1), lw=4, ec='royalblue')
plt.title("Histograma TVDI")
plt.xlabel('TVDI')
plt.ylabel('No. de pixels')
plt.tight_layout()

plt.show()

# Referências
DU, Lingtong et al. Comparison of two simulation methods of the temperature vegetation dryness index (TVDI) for drought monitoring in semi-arid regions of China. **Remote Sensing**, v. 9, n. 2, p. 177, 2017.

SANDHOLT, I.; RASMUSSEN, K.; ANDERSEN, J. A simple interpretation of the surface temperature/vegetation index space for assessment of surface moisture status. **Remote Sensing of environment**, v. 79, n. 2-3, p. 213-224, 2002.

SCHIRMBECK, L. W.; FONTANA, D. C.; SCHIRMBECK, J. Two approaches to calculate TVDI in humid subtropical climate of southern Brazil. **Scientia Agricola**, v. 75, n. 2, p. 111-120, 2018.

MENG, L. et al. The calculation of TVDI based on the composite time of pixel and drought analysis. **The International Archives of the Photogrammetry, Remote Sensing and Spatial Information Sciences**, v. 38, n. Part II, 2010.

# Perspectivas futuras


*   Aplicar máscara dos corpos d'água
*   Modificar o método para que o TVDI possa ser comparado ao longo do tempo e nas diferentes estações


